In [2]:
import urllib
from bs4 import BeautifulSoup

In [3]:
#url = "https://quiniwine.com/api/pub/wineReviews?wine_id=53ab5f2adab5f0020000001f"
wine_id = "53ab5f2adab5f0020000001f"
url = "https://quiniwine.com/api/pub/wineReviews?wine_id=%s" % wine_id
html = urllib.request.urlopen(url).read()

In [4]:
bs = BeautifulSoup(html, 'html.parser')

In [5]:
table = bs.find("table")

In [6]:
table

<table><tr class="eyeprop"><td class="property">EYE/15</td><td class="score">12</td><td class="text">Clear rim, Clear depth</td></tr><tr class="noseprop"><td class="property">NOSE/30</td><td class="score">23</td><td class="text">Citrus, Floral, Fruity, White Flowers, Tropical, Dried, Orange Blossom aromas</td></tr><tr class="mouthprop"><td class="property">MOUTH/30</td><td class="score">24</td><td class="text">Grapefruit, Melon, Orange Blossom, Floral, Citrus, Fruity, Spicy flavours, Fresh sweetness, Fresh acidity, Rich tannins, Mild alcohol</td></tr><tr class="finishprop"><td class="property">FINISH/15</td><td class="score">12</td><td class="text">Medium duration, Good quality, Early peaktime</td></tr><tr class="opinionprop"><td class="property">OPINION/10</td><td class="score">8</td><td class="text">Simple complexity, Pleasant interest, Expected typicity, Harmonious balance</td></tr></table>

In [7]:
scores = table.findAll("td", class_="score")
reviews = table.findAll("td", class_="text")

In [32]:
def texToInt(elem):
    
    return int(elem.text)

def assignScore(score_list):
    
    eye = texToInt(score_list[0])
    nose = texToInt(score_list[1])
    mouth = texToInt(score_list[2])
    finish = texToInt(score_list[3])
    opinion = texToInt(score_list[4])
    
    d = {"eye_score": eye, "nose_score": nose, "mouth_score": mouth, 
         "finish_score": finish, "opinion_score": opinion}

    return d

def assignReview(revew_list):
    
    eye = revew_list[0].text
    nose = revew_list[1].text
    mouth = revew_list[2].text
    finish = revew_list[3].text
    opinion = revew_list[4].text
    
    d = {"eye": eye, "nose": nose, "mouth": mouth, 
         "finish": finish, "opinion": opinion}
    
    return d

In [9]:
score_dict = assignScore(scores)
review_dict = assignReview(reviews)

In [33]:
def sum_dict(d):
    
    return sum(d.values())
    

In [11]:
score_total = sum_dict(score_dict)

In [12]:
import pandas as pd

In [13]:
t = open("wine_table_output2.txt")

In [14]:
table = pd.read_table("wine_table_output2.txt")

In [15]:
ids = table.drop_duplicates()[2:]

In [16]:
ids = ids.reset_index(drop=True)

In [17]:
len(ids)

2097

In [18]:
ids.columns = ["id"]

In [19]:
id_list = []

for i in range(len(ids)):
    id_list.append(ids['id'][i][1:])

In [20]:
id_list = id_list[0:-1]

In [21]:
id_list

['55585e69827e050300000007',
 '535975d9682acc02000074bb',
 '55a28134174e540300000014',
 '535977ba682acc0200008ff4',
 '58cec814d69ba3b478df074e',
 '535973d0682acc0200000359',
 '535a5f7a7dd3dc02000000b0',
 '56b0f490cb9d45030060cda1',
 '554250f1eb915703000026ed',
 '554250eceb91570300001852',
 '589e83edb604e44b1619adb1',
 '532b8b2e9175ea020000003a',
 '535973f4682acc0200003d11',
 '535a5f847dd3dc020000135c',
 '535974fc682acc0200006646',
 '535a5fbb7dd3dc0200003377',
 '53597867682acc0200009f51',
 '530b9a4e57428602000000b2',
 '55a3d108a08d0a0300000053',
 '535a684f7dd3dc0200008985',
 '554250e8eb91570300000cf5',
 '554250e7eb91570300000ab9',
 '545803bc1888f1020000000b',
 '59d527533392cc0e34ed84e1',
 '580e825fe12c5d7b522aff8b',
 '554250e8eb91570300000c89',
 '535975af682acc020000706c',
 '554250e5eb9157030000015e',
 '554250eceb91570300001cb1',
 '546159e04decd70200000026',
 '535a627d7dd3dc0200006db8',
 '535973e4682acc0200002fd1',
 '53642d146613f4020000002b',
 '56266f27d8ae270300000020',
 '5359777f682a

In [47]:
def create_review_dataset(wineId):
    base_url = "https://quiniwine.com/api/pub/wineReviews?wine_id=%s" % wineId
    html = urllib.request.urlopen(base_url).read()
    bs = BeautifulSoup(html, 'html.parser')
    
    if bs.find("table"):
        table = bs.find("table")
        scores = table.findAll("td", class_="score")
        reviews = table.findAll("td", class_="text")
    
        score_dict = assignScore(scores)
        review_dict = assignReview(reviews)
    
        reviews = score_dict.copy()
        reviews.update(review_dict)

        reviews["wine_id"] = wineId
        reviews["total_score"] = sum(score_dict.values())
        
        return reviews
    
    else:
        return {"wine_id": wineId, "eye": "", "eye_score": 0, 
                "nose": "", "nose_score": 0, "mouth": "",
                "mouth_score": 0, "finish": "", "finish_score": 0,
                "opinion": "", "opinion_score": 0, "total_score": 0}

In [44]:
df = pd.DataFrame(columns=["wine_id", "eye", "eye_score", "nose", "nose_score", "mouth", "mouth_score", "finish", "finish_score", "opinion", "opinion_score", "total_score"])

In [45]:
df.loc.__dict__

{'name': 'loc', 'ndim': 2, 'obj': Empty DataFrame
 Columns: [wine_id, eye, eye_score, nose, nose_score, mouth, mouth_score, finish, finish_score, opinion, opinion_score, total_score]
 Index: []}

In [26]:
id_list[3]

'535977ba682acc0200008ff4'

In [50]:
review_list = []
for i in id_list:
    review_list.append(create_review_dataset(i))

In [57]:
review_list

[{'eye': 'Clear rim, Translucent depth',
  'eye_score': 12,
  'finish': 'Long duration, Outstanding quality, Late peaktime',
  'finish_score': 13,
  'mouth': 'Tobacco, Sherry, Raspberry, Raisin, Prune, Cedar, Spicy flavours, Flat sweetness, Fresh acidity, Drying tannins, Mild alcohol',
  'mouth_score': 18,
  'nose': 'Sherry, Dust, Raspberry, Raisin, Prune, Spicy, Savoury aromas',
  'nose_score': 22,
  'opinion': 'Subtle complexity, Memorable interest, Expected typicity, Harmonious balance',
  'opinion_score': 8,
  'total_score': 73,
  'wine_id': '55585e69827e050300000007'},
 {'eye': '',
  'eye_score': 0,
  'finish': '',
  'finish_score': 0,
  'mouth': '',
  'mouth_score': 0,
  'nose': '',
  'nose_score': 0,
  'opinion': '',
  'opinion_score': 0,
  'total_score': 0,
  'wine_id': '535975d9682acc02000074bb'},
 {'eye': 'Clear rim, Clear depth',
  'eye_score': 11,
  'finish': 'Medium duration, Good quality, Middle peaktime',
  'finish_score': 11,
  'mouth': 'Berry, Strawberry, Raspberry fla

In [ ]:
review_list[3]

In [58]:
for id in ids.iterrows():
        print(id[1].values)

[' 55585e69827e050300000007']
[' 535975d9682acc02000074bb']
[' 55a28134174e540300000014']
[' 535977ba682acc0200008ff4']
[' 58cec814d69ba3b478df074e']
[' 535973d0682acc0200000359']
[' 535a5f7a7dd3dc02000000b0']
[' 56b0f490cb9d45030060cda1']
[' 554250f1eb915703000026ed']
[' 554250eceb91570300001852']
[' 589e83edb604e44b1619adb1']
[' 532b8b2e9175ea020000003a']
[' 535973f4682acc0200003d11']
[' 535a5f847dd3dc020000135c']
[' 535974fc682acc0200006646']
[' 535a5fbb7dd3dc0200003377']
[' 53597867682acc0200009f51']
[' 530b9a4e57428602000000b2']
[' 55a3d108a08d0a0300000053']
[' 535a684f7dd3dc0200008985']
[' 554250e8eb91570300000cf5']
[' 554250e7eb91570300000ab9']
[' 545803bc1888f1020000000b']
[' 59d527533392cc0e34ed84e1']
[' 580e825fe12c5d7b522aff8b']
[' 554250e8eb91570300000c89']
[' 535975af682acc020000706c']
[' 554250e5eb9157030000015e']
[' 554250eceb91570300001cb1']
[' 546159e04decd70200000026']
[' 535a627d7dd3dc0200006db8']
[' 535973e4682acc0200002fd1']
[' 53642d146613f4020000002b']
[' 56266f2

[' 535a5fb17dd3dc0200002cde']
[' 535975e5682acc02000079e5']
[' 535a5f867dd3dc02000015e8']
[' 535a5f897dd3dc0200001917']
[' 5359745e682acc0200005063']
[' 53597576682acc02000069b9']
[' 55df9b2bc70de6030000004f']
[' 53f5544b0719060200000029']
[' 554250eceb91570300001787']
[' 57ef35de58e3d48f1c8cf7dc']
[' 569eb7a7e88cc9120a496c08']
[' 569eb7a9e88cc9120a496c5a']
[' 56fb0680f9d8144837146c85']
[' 5717f5f5115cab5947f4e161']
[' 572d54258b4fbd5b302fd707']
[' 55d0e5442f033d03000002b5']
[' 56296a7ebac87b0300000002']
[' 569eb7a4e88cc9120a4969fd']
[' 569eb7a4e88cc9120a496a0a']
[' 569eb7a4e88cc9120a496a28']
[' 569eb7a5e88cc9120a496ae1']
[' 569eb7a5e88cc9120a496a98']
[' 569eb7a5e88cc9120a496ada']
[' 569eb7aae88cc9120a496c6f']
[' 5ab170fe65ef0a890b442c38']
[' 5359757d682acc0200006da1']
[' 535a6b6b7dd3dc0200009330']
[' 535a649b7dd3dc0200007d3f']
[' 535a6b6b7dd3dc020000932f']
[' 535a6b6b7dd3dc0200009331']
[' 535a62bf7dd3dc0200006f92']
[' 535a5f867dd3dc020000166f']
[' 535a68077dd3dc020000876f']
[' 569eb7a

[' 569eb7a4e88cc9120a496a00']
[' 569eb7a4e88cc9120a496a4e']
[' 569eb7a6e88cc9120a496b4f']
[' 556a194a72f2c90300000046']
[' 55a3d546a08d0a0300000062']
[' 55a80cc6e891cf030000035b']
[' 55cbfa407ad2fb030000000e']
[' 554250eceb91570300001d0b']
[' 554250edeb91570300001e34']
[' 554250edeb91570300001e65']
[' 554250eeeb91570300001efb']
[' 554250efeb915703000020f2']
[' 554250efeb915703000021df']
[' 59db55249255ae6a0a4a27bb']
[' 5aa3474e9d09eeb807fbec65']
[' 554250eceb91570300001bd2']
[' 535c99125211b40200000010']
[' 535973ec682acc0200003561']
[' 535a5f7f7dd3dc02000009ac']
[' 535a60397dd3dc0200004f62']
[' 535a61b37dd3dc0200006370']
[' 535a5fff7dd3dc020000446c']
[' 5465945a370c630200000051']
[' 549caf9fb26fbf0200000019']
[' 554250e8eb91570300000e35']
[' 575b853769d79f505b9152ed']
[' 56259f4329dc1f03000001b4']
[' 5657b25f1d426403008e0d7f']
[' 555394275cded60300000024']
[' 5579bdea3d3719030000004d']
[' 5ad77d38128caabb7db02834']
[' 535973d0682acc020000011e']
[' 535973d0682acc020000058c']
[' 535973d

In [59]:
#df.to_csv("wine_review.csv")

df2 = pd.DataFrame.from_dict(review_list)

In [61]:
df2
df3 = df2[["wine_id", "eye", "eye_score", "nose", "nose_score", "mouth", "mouth_score",
           "finish", "finish_score", "opinion", "opinion_score", "total_score"]]

In [63]:
df3.to_csv("wine_review.csv")